# Final project

1. Propose new classification, each group proposes its own </br>
2. Choose two zero shot models from https://huggingface.co/models?pipeline_tag=zero-shot-classification </br>
3. Calculate predictions for each model on Google Colab</br>
4. Calculate cross-tabulation for each prediction </br>
5. Compare results, are there any differences </br>
6. Write conclusions about difference, speed of computation etc.</br>
7. Write what the results tell us about the student opinions regarding courses and professors </br>
8. Are there any recommendations for universities' management </br>

In [1]:
import torch

In [2]:
#exetute this cell only in Google Colab
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 59.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import pipeline
import os
import torch
import numpy as np
import pandas as pd
import pickle

In [4]:
#exetute this cell only in Google Colab
from google.colab import files
import io

In [5]:
#get index for the maximum score from prediction in model distilbert-base-uncased-emotion
############################################################
def get_max_index(ld):
    lv = [d['score'] for d in ld]
    lvind = lv.index(max(lv))
    return lvind
############################################################

In [6]:
#get label with the maximum score for model distilbert-base-uncased-mnli
############################################################
def get_max_label(ld):
    val = ld['scores']
    val_ind = lv.index(max(lv))
    label = ld['labels'][val_ind]
    return label
############################################################

In [7]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [9]:
rmp = pd.read_csv("./rmp_small.csv") #for local drive

In [10]:
rmp

,review,score
0,"he can be a mean man sometimes, but just get o...",2
1,"He's fun, but he speeds through lectures and l...",2
2,Dr. Foster is an amazing teacher. One of my fa...,4
3,She is one of the greatest teacher I have ever...,4
4,Professor Bird was very easy and relaxed. If y...,3
...,...,...
97540,Charismatic and enthusiastic but that's it . H...,0
97541,"Great books - Dickens, Eliot, etc. A little d...",3
97542,Awesome professor. Doesnt stop explaining unti...,4
97543,"He has a really high standard, but grades fair...",4


In [11]:
sentences = list(rmp['review'])
labels = list(rmp['score'])

In [12]:
#remove nan from both lists
selnan = list()
for idx,name in enumerate(sentences):
    if name == '' or pd.isnull(name):
        selnan.append(idx)
sentences = [sentences[i] for i, e in enumerate(sentences) if i not in selnan]
labels = [labels[i] for i, e in enumerate(labels) if i not in selnan]
(len(sentences), len(labels))

(97545, 97545)

In [13]:
#do zero-shot classification
#check documentation https://huggingface.co/typeform/distilbert-base-uncased-mnli
classifier = pipeline("zero-shot-classification", 
                      model = "cross-encoder/nli-deberta-base",
                      device = 0 
)

Downloading:   0%|          | 0.00/975 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/531M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778 [00:00<?, ?B/s]

In [14]:
l1="enthusiastic student with high expectations"
l2="student with exceeded expectations"
l3="disappointed student with unmet expectations"
l4="indifferent or uninterested student with no expectations"
l5="other"
candidate_labels=[l1, l2, l3, l4, l5]

In [15]:
prediction = classifier(sentences, candidate_labels=candidate_labels)

In [28]:
prediction

{'labels': ['student with exceeded expectations',
  'enthusiastic student with high expectations',
  'disappointed student with unmet expectations',
  'other',
  'indifferent or uninterested student with no expectations'],
 'scores': [0.7994593977928162,
  0.14668115973472595,
  0.026990815997123718,
  0.02390293776988983,
  0.0029656251426786184],
 'sequence': 'He has a really high standard, but grades fairly.  You absolutely have to attend class, though.  There are many small details about the assignments that you can only get by being in the class and paying attention.'}

In [19]:
lv = prediction[90]['scores']
lv

[0.47799256443977356,
 0.4173375964164734,
 0.09112203121185303,
 0.008043189533054829,
 0.00550463330000639]

In [20]:
lvind = lv.index(max(lv))
lvind

0

In [22]:
l_max_labels = [get_max_label(val) for val in prediction]

In [23]:
df = pd.DataFrame(list(zip(l_max_labels, labels)), columns =['prof_opinion', 'score'])
df

,prof_opinion,score
0,other,2
1,enthusiastic student with high expectations,2
2,enthusiastic student with high expectations,4
3,other,4
4,other,3
...,...,...
97540,disappointed student with unmet expectations,0
97541,other,3
97542,other,4
97543,student with exceeded expectations,4


In [24]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result

score,0,1,2,3,4
prof_opinion,,,,,
disappointed student with unmet expectations,10419,5608,3395,2005,1221
enthusiastic student with high expectations,376,726,1863,5923,9810
indifferent or uninterested student with no expectations,1528,558,277,230,235
other,3009,2343,3690,6615,8980
student with exceeded expectations,659,1135,2984,8577,15379
